In [1]:
!pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="<API_KEY>"
os.environ["LANGCHAIN_PROJECT"]="RAG_ADVANCED"
os.environ['OPENAI_API_KEY'] = "<API_KEY>"


from langsmith import utils
utils.tracing_is_enabled()

True

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [4]:
#Load Documents
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/", ),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [5]:
#Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

#Embed
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

vectorestore = Chroma.from_documents(documents=splits, embedding = embedding, persist_directory="./db003")

#Retriever
retriever = vectorestore.as_retriever()

In [9]:
from langchain.prompts import ChatPromptTemplate

#RAG-Fusion: Related

template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to :{question}\n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [11]:
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)

generate_queries = (
    prompt_rag_fusion
    | llm
    | StrOutputParser()
    | (lambda x:x.split("\n"))
)

In [32]:
generate_queries.invoke("What is task decomposition for LLM agents?")

['1. How does task decomposition work in large language model (LLM) agents?',
 '2. Benefits of task decomposition for LLM agents in AI applications',
 '3. Examples of task decomposition techniques used in LLM agents',
 '4. Comparing task decomposition methods for LLM agents and traditional AI systems']

In [40]:
retrieval_chain_rag_fusion_test = generate_queries | retriever.map()

result = retrieval_chain_rag_fusion_test.invoke("What is task decomposition for LLM agents?")

for docs in result:
  for rank, doc in enumerate(docs):
    print(rank, dumps(doc))

0 {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"metadata": {"source": "https://lilianweng.github.io/posts/2023-06-23-agent/"}, "page_content": "The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.", "type": "Document"}}
1 {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"metadata": {"source": "https://lilianweng.github.io/posts/2023-06-23-agent/"}, "page_content": "Task decomposition can be done (1) by LLM with simple prompting like \"Steps for XYZ.\\n1.\", \"What are the subgoals for achieving XYZ?\", (2) by using task-specific instructions; e.g. \"Write a story outline.\" for writing a novel, or (3) with human inputs.", "type": "Document"}}
2

In [31]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results:list[list], k=60):
  #initialize the dictionary to hold fused scores for each unique document
  fused_scores = {}

  for docs in results:
    for rank, doc in enumerate(docs):
      #convert the document to a string to use as a key in dictionary
      doc_str = dumps(doc)

      # if the document is not yet in the dictionary, add it with an initial score of 0
      if doc_str not in fused_scores:
        fused_scores[doc_str] = 0

      fused_scores[doc_str] = fused_scores[doc_str] + 1 / (rank + k)

  reranked_results = [(loads(doc), score) for doc, score in sorted(fused_scores.items(), key = lambda x: x[1], reverse= True)]
  return reranked_results

question = "What is task decomposition for LLM agents?"

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

docs = retrieval_chain_rag_fusion.invoke({"question":question})
len(docs)

9

In [29]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

#RAG
template = """Answer the following question based on this context:

{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context":retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents involves breaking down large tasks into smaller, manageable subgoals. This process enables the efficient handling of complex tasks by allowing the agent to focus on individual components rather than the entire task at once. Task decomposition can be achieved through various methods, including:\n\n1. Simple prompting, such as asking for steps or subgoals related to a specific task.\n2. Task-specific instructions that guide the agent on how to approach a particular type of task (e.g., writing a story outline).\n3. Human inputs that provide additional context or direction for the decomposition process.\n\nOverall, task decomposition is a critical aspect of planning and executing tasks effectively within an LLM-powered autonomous agent system.'